In [18]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from datasets import TripletAudio
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from recall import Recall
from torch.optim import lr_scheduler
from trainer import fit
from recall import Recall
import itertools
from tensorboardX import SummaryWriter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
#define hyperparams
K, MAX_CLOSE_NEG, P_STRONG_NEG = 5, 15, 1
BATCH_SIZE = 128
INPUT_D, OUTPUT_D = 192, 128
MARGIN, N_EPOCHS, LOG_INT, N_RECALL_CAND = 0.5, 20, 100, 25
N_RECALL_SAMPLES = 1000
# hyperparams = [P_STRONG_NEGS]

In [115]:
#computes the recall on datasets using global variables such as model and recall
def measure_recall(datasets):
    outputs = []
    for d in datasets:
        batch_indicies = np.random.choice(list(range(0, d.get_dataset().shape[0])), N_RECALL_SAMPLES, False)
        queries = d.get_dataset()[batch_indicies]
        knns_of_batch = d.get_knn().iloc[batch_indicies, :]
        outputs.append(recall.calculate(d.get_dataset(), model.embedding_net, queries, knns_of_batch, False))
    return [np.mean(x) for x in outputs]

In [157]:
while True:
#     for P_STRONG_NEG in P_STRONG_NEGS: #itertools.product(*hyperparams):
    #setup datasets
    triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, P_STRONG_NEG)
    triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, P_STRONG_NEG)
    triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    #define model 
    anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
    embedding_net = EmbeddingNet(anchor_net)
    model = TripletNet(embedding_net)
    loss_fn = TripletLoss(MARGIN)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1, last_epoch=-1)
    recall = Recall(N_RECALL_CAND, K)
    #run the model
    train_loss, val_loss = fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)
    #measure recall
    train_recall, val_recall = measure_recall([triplet_train_dataset, triplet_test_dataset])
    #write to tensorboard
    with SummaryWriter() as w:
        w.add_hparams(
            {'LR': LEARNING_RATE, 'BSIZE': BATCH_SIZE, 'N_RECALL_S': N_RECALL_SAMPLES, 'N_RECALL_CAND': N_RECALL_CAND,
                 'CLOSE_NEG': MAX_CLOSE_NEG, 'P_STRONG_NEG': P_STRONG_NEG, 'OUTPUT_D': OUTPUT_D},
            {'TRAIN_L': train_loss, 'VAL_L': val_loss, 'TRAIN_RECALL': train_recall, 'VAL_RECALL': val_recall})

initialising model biases
done
Train: [0/48048 (0%)]	Loss: 15.933144
Train: [12800/48048 (27%)]	Loss: 16.671302
Train: [25600/48048 (53%)]	Loss: 15.290250
Train: [38400/48048 (80%)]	Loss: 15.189563
Epoch: 1/20. Train set: Average loss: 15.4895
Epoch: 1/20. Validation set: Average loss: 16.9150
Train: [0/48048 (0%)]	Loss: 11.316054
Train: [12800/48048 (27%)]	Loss: 13.119747
Train: [25600/48048 (53%)]	Loss: 12.338122
Train: [38400/48048 (80%)]	Loss: 11.385098
Epoch: 2/20. Train set: Average loss: 11.8998
Epoch: 2/20. Validation set: Average loss: 11.9876
Train: [0/48048 (0%)]	Loss: 11.608500
Train: [12800/48048 (27%)]	Loss: 8.862409
Train: [25600/48048 (53%)]	Loss: 8.076705
Train: [38400/48048 (80%)]	Loss: 7.750642
Epoch: 3/20. Train set: Average loss: 7.9997
Epoch: 3/20. Validation set: Average loss: 8.0638
Train: [0/48048 (0%)]	Loss: 6.586010
Train: [12800/48048 (27%)]	Loss: 6.257767
Train: [25600/48048 (53%)]	Loss: 5.837694
Train: [38400/48048 (80%)]	Loss: 5.068677
Epoch: 4/20. Train 

KeyboardInterrupt: 

### Online Selection

In [ ]:
#prepare data 
# %load_ext autoreload
%autoreload 2
import torch
from datasets import AudioTrainDataset, AudioTestDataset
from datasets import BalancedBatchSampler

K = 5
train_dataset = AudioTrainDataset(K)
test_dataset = AudioTestDataset(K)

train_batch_sampler = BalancedBatchSampler(train_dataset)
test_batch_sampler = BalancedBatchSampler(test_dataset)

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler)

In [ ]:
# Set up the network and training parameters
from networks import EmbeddingNet, AnchorNet
from losses import OnlineTripletLoss
from utils import SemihardNegativeTripletSelector, HardestNegativeTripletSelector
from metrics import AverageNonzeroTripletsMetric
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 50

#define model
anchor_net = AnchorNet(train_dataset.data, INPUT_D, OUTPUT_D)
model = EmbeddingNet(anchor_net)
loss_fn = OnlineTripletLoss(MARGIN, SemihardNegativeTripletSelector(MARGIN, train_dataset.KNN))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [ ]:
#run model
from trainer import fit
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT, metrics=[AverageNonzeroTripletsMetric()])